# Graph isomorphism problem QUBO solved with QAOA

In [1]:
import numpy as np

## Qubo function

In [2]:
def create_qubo(E1,E2,vertices,p):
    Q = np.zeros((vertices*vertices, vertices*vertices))
    
    # Constraint 1: penalty if several mappings from same source
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i*vertices+j,i*vertices+k]=p 

    # Constaint 2: penalty if several mappings to same target
    for i in range(vertices): 
        for j in range(vertices): 
            for k in range(j+1,vertices): 
                Q[i+vertices*j,i+vertices*k]=p 
                
    # Constraint 3: -1 for each succesfully mapped edge: (x1,y1) -> (x2,y2) 
    #    two possible mappings: (x1->x2, y1->y2) or (x1->y2,y1->x2)
    for e1 in E1: 
        for e2 in E2: 
            Q[e1[0]*vertices+e2[0], e1[1]*vertices+e2[1]] -= 1
            Q[e1[0]*vertices+e2[1], e1[1]*vertices+e2[0]] -= 1
            
    # All quadratic coefficients in lower triangle to upper triangle
    for i in range(vertices): 
        for j in range(i):
            Q[j,i] += Q[i,j]
            Q[i,j] = 0
    return Q

In [18]:
def result_info(result, offset, min_e, labels):
    le=int(np.real(result.best_measurement['value']+offset))
    print('Lowest energy should be:',min_e)
    print('Lowest energy was:',le)
    results = []
    if min_e!=le:
        print('Graphs are NOT isomorphic')
    else:
        print('Graphs are isomorphic')
        m = ''
        bs = result.best_measurement['bitstring']
        for i in range(len(bs)):
            if bs[len(bs)-i-1]=='1':
                m += str(labels[i])+', '
        print('Mapping: '+m)

## Graph and QUBO

In [4]:
vertices = 3
E1 = np.array([(0, 1), (1, 2)])
E2 = np.array([(0, 1), (0, 2)]) 

labels = {}
for i in range(vertices):
    for j in range(vertices):
        labels[i*vertices+j] = (i,j)          
p = len(E1)

Q = create_qubo(E1,E2,vertices,p)
qubosize= vertices*vertices

## Hamiltonian $H_p$

In [5]:
from qiskit.quantum_info import SparsePauliOp

oplist=[]
offset = 0
for i in range(qubosize):
    if Q[i,i]!=0:
        oplist.append(('Z',i,-Q[i,i]/2))
    else:
        offset += 1/2
for i in range(qubosize):
    for j in range(i+1,qubosize):
        if Q[i,j]!=0:
            oplist.append(('ZZ',[i,j],Q[i,j]/4))
            oplist.append(('Z',[i],-Q[i,j]/4))
            oplist.append(('Z',[j],-Q[i,j]/4))
        else:
            offset += 1/4
                          
H_p = SparsePauliOp.from_sparse_list(oplist, num_qubits=qubosize).simplify()

## Run with local computer

In [9]:
from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

sampler = Sampler()
qaoa = QAOA(sampler, COBYLA(), reps=2, initial_point=[0.0, 0.0, 0.0, 0.0])
result = qaoa.compute_minimum_eigenvalue(H_p) # This includes automatically mixer operator

In [19]:
result_info(result, offset, -p, labels)

Lowest energy should be: -2
Lowest energy was: -2
Graphs are isomorphic
Mapping: (0, 2), (1, 0), (2, 1), 


Correct mapping (0,1),(1,0),(2,2) and (0,2),(1,0),(2,1)

In [20]:
print('Optimal parameters:',result.optimal_point)
print('Optimal value:',result.optimal_value)
print('Number of optimizer evaluations:',result.optimizer_evals)
print('Optimizer result:',result.optimizer_result)
print('Time (s):',result.optimizer_time)
print('Eigenvalue:',result.eigenvalue)
print('Number of cost optimizer evaluations:',result.cost_function_evals)

Optimal parameters: [0. 0. 0. 0.]
Optimal value: 0.0
Number of optimizer evaluations: None
Optimizer result: {   'fun': 0.0,
    'jac': None,
    'nfev': 25,
    'nit': None,
    'njev': None,
    'x': array([0., 0., 0., 0.])}
Time (s): 1.5501558780670166
Eigenvalue: 0.0
Number of cost optimizer evaluations: 25


## Run with IBM simulator

In [21]:
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.get_backend("ibmq_qasm_simulator")
sampler = Sampler(backend=backend)

In [22]:
qaoa = QAOA(sampler, COBYLA(), reps=2, initial_point=[0.0, 0.0, 0.0, 0.0])
result = qaoa.compute_minimum_eigenvalue(H_p) # This includes automatically mixer operator

In [23]:
result_info(result, offset, -p, labels)

Lowest energy should be: -2
Lowest energy was: -2
Graphs are isomorphic
Mapping: (0, 2), (1, 0), (2, 1), 


In [24]:
print('Optimal parameters:',result.optimal_point)
print('Optimal value:',result.optimal_value)
print('Number of optimizer evaluations:',result.optimizer_evals)
print('Optimizer result:',result.optimizer_result)
print('Time (s):',result.optimizer_time)
print('Eigenvalue:',result.eigenvalue)
print('Number of cost optimizer evaluations:',result.cost_function_evals)

Optimal parameters: [0.97382635 1.54282181 0.30400524 0.86221677]
Optimal value: -6.671750000000005
Number of optimizer evaluations: None
Optimizer result: {   'fun': -6.671750000000005,
    'jac': None,
    'nfev': 65,
    'nit': None,
    'njev': None,
    'x': array([0.97382635, 1.54282181, 0.30400524, 0.86221677])}
Time (s): 140.02241730690002
Eigenvalue: -6.671750000000005
Number of cost optimizer evaluations: 65


## Larger graphs

In [26]:
vertices = 7
E1 = np.array([(0, 1), (0, 2), (0, 3), (2, 4), (3, 5), (5, 6)])
E2 = np.array([(0, 1), (0, 2), (0, 3), (2, 4), (3, 5), (0, 6)])# last number difference
E3 = np.array([(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]) # very different

perm={0:2, 1:1, 2:3, 3:4, 4:0, 5:6, 6:5}
E4 = np.array(E1, copy=True) 
for i in range(len(E2)):
    E4[i]=(perm[E1[i][0]],perm[E1[i][1]])

perm2={0:2, 1:1, 2:0, 3:3, 4:4, 5:5, 6:6}
E5 = np.array(E1, copy=True) 
for i in range(len(E3)):
    E5[i]=(perm2[E1[i][0]],perm2[E1[i][1]])

labels = {}
for i in range(vertices):
    for j in range(vertices):
        labels[i*vertices+j] = (i,j)
            
p = len(E1)
print('Penalty:',p)

Penalty: 6


In [29]:
Q = create_qubo(E1,E2,vertices,p)
qubosize= vertices*vertices

In [30]:
oplist=[]
offset = 0
for i in range(qubosize):
    if Q[i,i]!=0:
        oplist.append(('Z',i,-Q[i,i]/2))
    else:
        offset += 1/2
for i in range(qubosize):
    for j in range(i+1,qubosize):
        if Q[i,j]!=0:
            oplist.append(('ZZ',[i,j],Q[i,j]/4))
            oplist.append(('Z',[i],-Q[i,j]/4))
            oplist.append(('Z',[j],-Q[i,j]/4))
        else:
            offset += 1/4
                          
H_p = SparsePauliOp.from_sparse_list(oplist, num_qubits=qubosize).simplify()

In [31]:
from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

sampler = Sampler()
qaoa = QAOA(sampler, COBYLA(), reps=2, initial_point=[0.0, 0.0, 0.0, 0.0])
result = qaoa.compute_minimum_eigenvalue(H_p)
result_info(result, offset, -p, labels)

MemoryError: Unable to allocate 8.00 PiB for an array with shape (562949953421312,) and data type complex128

In [32]:
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.get_backend("ibmq_qasm_simulator")
sampler = Sampler(backend=backend)

qaoa = QAOA(sampler, COBYLA(), reps=2, initial_point=[0.0, 0.0, 0.0, 0.0])
result = qaoa.compute_minimum_eigenvalue(H_p) # This includes automatically mixer operator
result_info(result, offset, -p, labels)

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


RuntimeJobMaxTimeoutError: 'RAN TOO LONG'